In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import cv2
import torch
import torch.functional as F
import torch.nn as nn
from torch.optim import Adam
from time import time, sleep

from utils import *
from custom_pca import custom_pca
from video_loader import VideoLoader
from autoencoders import *

### Example of video transformation

In [2]:
model = custom_pca()

video = VideoLoader('data/sample20s.mp4', grayscale=True)
t1 = time()
frames_rand = video.get_random_frames(0.6)
model.fit(frames_rand)
t2 = time()
reconstructed = []
for j, frames in enumerate(video):
    reconstructed.append(model.inverse_transform(model.transform(frames), shape=(video.height, video.width)))
reconstructed = np.vstack(reconstructed)
t3 = time()
print(reconstruction_error(video.get_all_frames(), reconstructed))
t4 = time()

print('Fitting time:', t2-t1)
print('Transform:', t3-t2)
print('Error calculation:', t4-t3)

Shapes don't match
-1
Fitting time: 17.691691398620605
Transform: 44.788443088531494
Error calculation: 1.6141698360443115


### Dimensionality reduction using autoencoders

#### Consecutive load

Number of parameters for 16 components: 56466
Average loss per pixel at epoch 1: 57.70232963562012
Average loss per pixel at epoch 2: 49.1624118141506
Average loss per pixel at epoch 3: 44.3708496923032
Average loss per pixel at epoch 4: 43.27957932845406
Average loss per pixel at epoch 5: 43.63873929562776
Error after 5 epochs: 40.391567
Time taken for fitting with 6 epochs: 339.867027759552 sec
Time taken to transform: 132.30144000053406 sec


In [13]:
video = VideoLoader('data/big_sample_256.mp4', grayscale=True, duration=60*5, skip_frame=4)
num_epoch = 5
ncomp = 16
network = BasicAutoEncoder(1, ncomp)
optimizer = Adam(network.parameters(), lr=0.0005)

print(f'Number of parameters for {network.ncomp} components:', sum([p.numel() for p in network.parameters()]))
losses = []
networks = []
start = time()
for i in range(num_epoch):
    batch_losses = []
    for x in video:
        x = torch.from_numpy(x).float().unsqueeze(1)
        x_low, shape = network.transform(x)
        x_reconstructed = network.inverse_transform(x_low, shape)
        batch_loss = crit(x_reconstructed, x)
        batch_losses.append(batch_loss.item())
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
    losses.append(np.mean(batch_losses))
    print(f'Average loss per pixel at epoch {i+1}:', losses[-1])

fitting_time = time() - start

video.randit = video.skip_frame = 0
start = time()
reconstructed = []
for x in video:
    x = torch.from_numpy(x).float().unsqueeze(1)
    x_low, shape = network.transform(x)
    reconstructed.append(network.inverse_transform(x_low, shape).squeeze(1).detach().numpy())
reconstructed = np.vstack(reconstructed)
transform_time = time() - start

exact_loss = reconstruction_error(reconstructed,
                                  video.get_all_frames())
print("Error after %s epochs:" % num_epoch, exact_loss)
      
print(f'Time taken for fitting with {num_epoch+1} epochs:', fitting_time, 'sec')
print('Time taken to transform:', transform_time, 'sec')
losses = np.array(losses)
    
plot(range(1, num_epoch+1), [losses])

In [14]:
write_video('data/big_sample_net_16_consecutive.mp4', reconstructed, video.width, video.height, video.fps, grayscale=True)

#### Random loading

In [4]:
video = VideoLoader('data/big_sample_256.mp4', grayscale=True, duration=60*5, skip_frame=4, randit=True)
num_epoch = 20
ncomp = 16
network = BasicAutoEncoder(1, ncomp)
optimizer = Adam(network.parameters(), lr=0.0005)


print(f'Number of parameters for {network.ncomp} components:', sum([p.numel() for p in network.parameters()]))
losses = []
networks = []
start = time()
for i in range(num_epoch):
    batch_losses = []
    num_in_batch = []
    for x in video:
        num_in_batch.append(x.shape[0])
        x = torch.from_numpy(x).float().unsqueeze(1)
        x_low, shape = network.transform(x)
        x_reconstructed = network.inverse_transform(x_low, shape)
        batch_loss = crit(x_reconstructed, x)
        batch_losses.append(batch_loss.item())
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
    losses.append(np.mean(batch_losses))
    print(f'Average loss per pixel at epoch {i+1}:', losses[-1])

fitting_time = time() - start

video.randit = video.skip_frame = 0
start = time

reconstructed = []
for x in video:
    x = torch.from_numpy(x).float().unsqueeze(1)
    x_low, shape = network.transform(x)
    reconstructed.append(network.inverse_transform(x_low, shape).squeeze(1).detach().numpy())
reconstructed = np.vstack(reconstructed)
transform_time = time() - start

exact_loss = reconstruction_error(reconstructed,
                                  video.get_all_frames())
print("Error after %s epochs:" % num_epoch, exact_loss)
      
print(f'Time taken for fitting with {num_epoch+1} epochs:', fitting_time, 'sec')
print('Time taken to transform:', transform_time, 'sec')
losses = np.array(losses)
    
plot(range(1, num_epoch+1), [losses])

In [12]:
write_video('data/big_sample_net_16.mp4', reconstructed, video.width, video.height, video.fps, grayscale=True)

### Further tests

#### 100 components

Error after 20 epochs: 23.887798
Time taken for fitting with 21 epochs: 1280.099949836731 sec
Time taken to transform: 244.48246788978577 sec


In [7]:
video = VideoLoader('data/big_sample_256.mp4', grayscale=True, duration=60*5, skip_frame=4, randit=True)
num_epoch = 20
ncomp = 100
network = BasicAutoEncoder(1, ncomp)
optimizer = Adam(network.parameters(), lr=0.0005)


print(f'Number of parameters for {network.ncomp} components:', sum([p.numel() for p in network.parameters()]))
losses = []
networks = []
start = time()
for i in range(num_epoch):
    batch_losses = []
    num_in_batch = []
    for x in video:
        num_in_batch.append(x.shape[0])
        x = torch.from_numpy(x).float().unsqueeze(1)
        x_low, shape = network.transform(x)
        x_reconstructed = network.inverse_transform(x_low, shape)
        batch_loss = crit(x_reconstructed, x)
        batch_losses.append(batch_loss.item())
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
    losses.append(np.mean(batch_losses))
    print(f'Average loss per pixel at epoch {i+1}:', losses[-1])

fitting_time = time() - start

video.randit = video.skip_frame = 0
start = time()

reconstructed = []
for x in video:
    x = torch.from_numpy(x).float().unsqueeze(1)
    x_low, shape = network.transform(x)
    reconstructed.append(network.inverse_transform(x_low, shape).squeeze(1).detach().numpy())
reconstructed = np.vstack(reconstructed)
#transform_time = time() - start

exact_loss = reconstruction_error(reconstructed,
                                  video.get_all_frames())
print("Error after %s epochs:" % num_epoch, exact_loss)
      
print(f'Time taken for fitting with {num_epoch+1} epochs:', fitting_time, 'sec')
print('Time taken to transform:', transform_time, 'sec')
losses = np.array(losses)
    
plot(range(1, num_epoch+1), [losses])
exact_loss = reconstruction_error(reconstructed,
                                  video.get_all_frames())
print("Error after %s epochs:" % num_epoch, exact_loss)
      
print(f'Time taken for fitting with {num_epoch+1} epochs:', fitting_time, 'sec')
print('Time taken to transform:', transform_time, 'sec')
losses = np.array(losses)
    
plot(range(1, num_epoch+1), [losses])

#### Standardize the video

Number of parameters for 100 components: 57285
Average loss per pixel at epoch 1: 0.888941871083301
Average loss per pixel at epoch 2: 0.8139310064523116
Average loss per pixel at epoch 3: 0.7679049139437468
Average loss per pixel at epoch 4: 0.7272463389064955
Average loss per pixel at epoch 5: 0.6884975018708602
Average loss per pixel at epoch 6: 0.6543531210526176
Average loss per pixel at epoch 7: 0.6242800406787706
Average loss per pixel at epoch 8: 0.612094635548799
Average loss per pixel at epoch 9: 0.5807123806165613
Average loss per pixel at epoch 10: 0.5756290062614109
Average loss per pixel at epoch 11: 0.5531931364017985
Average loss per pixel at epoch 12: 0.5453619024027949
Average loss per pixel at epoch 13: 0.5325381380060444
Average loss per pixel at epoch 14: 0.5410454519416975
Average loss per pixel at epoch 15: 0.5258791809496672
Average loss per pixel at epoch 16: 0.5264319660870925
Average loss per pixel at epoch 17: 0.5060231905916462
Average loss per pixel at epo

Error after 20 epochs: 24.381197
Time taken for fitting with 21 epochs: 1638.3017599582672 sec
Time taken to transform: 126.21297097206116 sec


In [5]:
video = VideoLoader('data/big_sample_256.mp4', grayscale=True, duration=60*5, skip_frame=4, randit=True)


mean, std = np.mean(video.get_all_frames()), np.std(video.get_all_frames())


num_epoch = 20
ncomp = 100
network = BasicAutoEncoder(1, ncomp)
optimizer = Adam(network.parameters(), lr=0.0005)


print(f'Number of parameters for {network.ncomp} components:', sum([p.numel() for p in network.parameters()]))
losses = []
networks = []
start = time()
for i in range(num_epoch):
    batch_losses = []
    num_in_batch = []
    for x in video:
        num_in_batch.append(x.shape[0])
        x = torch.from_numpy(x).float().unsqueeze(1)
        x = (x-mean) / std
        x_low, shape = network.transform(x)
        x_reconstructed = network.inverse_transform(x_low, shape)
        batch_loss = crit(x_reconstructed, x)
        batch_losses.append(batch_loss.item())
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
    losses.append(np.mean(batch_losses))
    print(f'Average loss per pixel at epoch {i+1}:', losses[-1])

fitting_time = time() - start

video.randit = video.skip_frame = 0
start = time()

reconstructed = []
for x in video:
    x = torch.from_numpy(x).float().unsqueeze(1)
    x_low, shape = network.transform(x)
    x = (x-mean) / std
    reconstructed.append(network.inverse_transform(x_low, shape).squeeze(1).detach().numpy())
reconstructed = np.vstack(reconstructed)
transform_time = time() - start

exact_loss = reconstruction_error(reconstructed*std + mean,
                                 video.get_all_frames())
print("Error after %s epochs:" % num_epoch, exact_loss)
      
print(f'Time taken for fitting with {num_epoch+1} epochs:', fitting_time, 'sec')
print('Time taken to transform:', transform_time, 'sec')
losses = np.array(losses)
    
plot(range(1, num_epoch+1), [losses])

#### New autoencoder

In [ ]:
video = VideoLoader('data/big_sample_256.mp4', grayscale=True, duration=60*5, skip_frame=4, randit=True, batch_size=32)
num_epoch = 20
network = ShallowAutoEncoder(1)
optimizer = Adam(network.parameters(), lr=0.0005)


print(f'Number of parameters:', sum([p.numel() for p in network.parameters()]))
losses = []
networks = []
start = time()
for i in range(num_epoch):
    batch_losses = []
    num_in_batch = []
    for x in video:
        print(x.shape)
        num_in_batch.append(x.shape[0])
        x = torch.from_numpy(x).float().unsqueeze(1)
        x_low, shape = network.transform(x)
        print(shape)
        x_reconstructed = network.inverse_transform(x_low, shape)
        batch_loss = crit(x_reconstructed, x)
        batch_losses.append(batch_loss.item())
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
    losses.append(np.mean(batch_losses))
    print(f'Average loss per pixel at epoch {i+1}:', losses[-1])

fitting_time = time() - start

video.randit = video.skip_frame = 0
start = time()

reconstructed = []
for x in video:
    x = torch.from_numpy(x).float().unsqueeze(1)
    x_low, shape = network.transform(x)
    reconstructed.append(network.inverse_transform(x_low, shape).squeeze(1).detach().numpy())
    break
reconstructed = np.vstack(reconstructed)
#transform_time = time() - start

exact_loss = reconstruction_error(reconstructed,
                                  video.get_all_frames())
print("Error after %s epochs:" % num_epoch, exact_loss)
      
print(f'Time taken for fitting with {num_epoch+1} epochs:', fitting_time, 'sec')
print('Time taken to transform:', transform_time, 'sec')
losses = np.array(losses)
    
plot(range(1, num_epoch+1), [losses])
exact_loss = reconstruction_error(reconstructed,
                                  video.get_all_frames())
print("Error after %s epochs:" % num_epoch, exact_loss)
      
print(f'Time taken for fitting with {num_epoch+1} epochs:', fitting_time, 'sec')
print('Time taken to transform:', transform_time, 'sec')
losses = np.array(losses)
    
plot(range(1, num_epoch+1), [losses])

Number of parameters: 29317
